Deep Residual Learning for Image Recognition

In [2]:
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras
import tensorflow.keras.layers as layers

In [11]:
def Identity_shortcut(x,filters,strides = 1):
    shortcut = x
    x = layers.Conv2D(filters = filters, kernel_size = 3, strides = strides, padding = 'same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    x = layers.Conv2D(filters = filters, kernel_size = 3, padding = 'same')(x)
    x = layers.BatchNormalization()(x)
    if strides != 1:
        shortcut = layers.MaxPool2D(pool_size = 2, strides=2)(shortcut)
        shortcut = tf.pad(shortcut, tf.constant([[0, 0,], [0, 0,], [0, 0,], [0, filters//2]]), "CONSTANT")
    x = layers.Add()([shortcut,x])
    x = layers.ReLU()(x)
    return x

def Projection_shortcut(x,filters):
    shortcut = x
    x = layers.Conv2D(filters = filters, kernel_size = 3, strides = 2, padding = 'same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    x = layers.Conv2D(filters = filters, kernel_size = 3, padding = 'same')(x)
    x = layers.BatchNormalization()(x)
    
    shortcut = layers.Conv2D(filters = filters, kernel_size = 1, strides = 2, 
                            padding = 'same')(shortcut)
    shortcut = layers.BatchNormalization()(shortcut)
    
    x = layers.Add()([shortcut,x])
    x = layers.ReLU()(x)
    return x

def resnet_20():
    inputs = layers.Input(shape=(32,32,3))
    x = layers.Conv2D(filters = 16, kernel_size = 3,strides = 1,padding = 'same')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    #x = layers.MaxPooling2d(pool_size = (3,3), strides = 2, padding = 'same')
    
    x = Identity_shortcut(x,16)
    x = Identity_shortcut(x,16)
    x = Identity_shortcut(x,16)
    
    x = Identity_shortcut(x,32,2)
    x = Identity_shortcut(x,32)
    x = Identity_shortcut(x,32)
    
    x = Identity_shortcut(x,64,2)
    x = Identity_shortcut(x,64)
    x = Identity_shortcut(x,64)
    
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Flatten()(x)
    outputs = layers.Dense(10, activation='softmax')(x)
    
    model = keras.Model(inputs=inputs,outputs=outputs)
    
    model.compile(optimizer = 'adam',loss = 'sparse_categorical_crossentropy',
        metrics=['accuracy'])
    
    return model

In [13]:
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard
import datetime
import os

(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()

model = resnet_20() # or create_plain_net()
model.summary()

timestr = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
name = 'cifar-10_res_net_20-'+timestr 

checkpoint_path = "checkpoints/"+name+"/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)
os.system('mkdir {}'.format(checkpoint_dir))

# save model after each epoch
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1
)
tensorboard_callback = TensorBoard(
    log_dir='tensorboard_logs/'+name,
    histogram_freq=1
)

model.fit(
    x=x_train,
    y=y_train,
    epochs=20,
    verbose=1,
    validation_data=(x_test, y_test),
    batch_size=128,
    callbacks=[cp_callback, tensorboard_callback]
)

170500096/170498071 [==============================] - 27s 0us/step
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv2d_47 (Conv2D)              (None, 32, 32, 16)   448         input_6[0][0]                    
__________________________________________________________________________________________________
batch_normalization_47 (BatchNo (None, 32, 32, 16)   64          conv2d_47[0][0]                  
__________________________________________________________________________________________________
re_lu_44 (ReLU)                 (None, 32, 32, 16)   0           batch_normalization_47[0][0]     
________________________

Epoch 1/20
391/391 [==============================] - 293s 742ms/step - loss: 1.6217 - accuracy: 0.4260 - val_loss: 1.7203 - val_accuracy: 0.4745

Epoch 00001: saving model to checkpoints/cifar-10_res_net_20-20210106-155324\cp-0001.ckpt
INFO:tensorflow:Assets written to: checkpoints/cifar-10_res_net_20-20210106-155324\cp-0001.ckpt\assets
Epoch 2/20
391/391 [==============================] - 284s 726ms/step - loss: 0.9443 - accuracy: 0.6642 - val_loss: 1.2758 - val_accuracy: 0.5534

Epoch 00002: saving model to checkpoints/cifar-10_res_net_20-20210106-155324\cp-0002.ckpt
INFO:tensorflow:Assets written to: checkpoints/cifar-10_res_net_20-20210106-155324\cp-0002.ckpt\assets
Epoch 3/20
391/391 [==============================] - 285s 729ms/step - loss: 0.7329 - accuracy: 0.7451 - val_loss: 1.0390 - val_accuracy: 0.6497

Epoch 00003: saving model to checkpoints/cifar-10_res_net_20-20210106-155324\cp-0003.ckpt
INFO:tensorflow:Assets written to: checkpoints/cifar-10_res_net_20-20210106-155324\

Total params가 271,786인데 논문에는 0.27M으로 약 270000이다. 재현이 잘 되었다.

In [15]:
%load_ext tensorboard
%tensorboard --logdir logs

ERROR: Timed out waiting for TensorBoard to start. It may still be running as pid 15316.